In [ ]:
import multiprocessing as mp
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import math

def count_nonnan(row):
    return sum(1 for c in row[1:] if not math.isnan(c))

TRAIN_PATH = './cut/full/train'
VALIDATION_PATH = './cut/full/val'

sources = {
    'Daily': TRAIN_PATH + '/Daily.csv',
    'Hourly': TRAIN_PATH + '/Hourly.csv',
    'Monthly': TRAIN_PATH + '/Monthly.csv',
    'Quarterly': TRAIN_PATH + '/Quarterly.csv',
    'Weekly': TRAIN_PATH + '/Weekly.csv',
    'Yearly': TRAIN_PATH + '/Yearly.csv',
}

def cut(params):
    print()
    i, (sp, path) = params
    
    info = pd.read_csv('./data/M4-info.csv')
    sp_info = info.loc[info['SP'] == sp]
    
    timeseries = pd.read_csv(path)
    test = []
    short = 0
    for i in tqdm(range(timeseries.shape[0]), position=i, desc=sp):
        horizon = sp_info.iloc[i]['Horizon']
        length = count_nonnan(timeseries.iloc[i])
        if (length < horizon * 2):
            short += 1
        test.append(timeseries.iloc[i, (length-horizon+1):(length+1)])
        timeseries.iloc[i, (length-horizon+1):(length+1)] = np.nan
        
        
    os.makedirs(VALIDATION_PATH + '/train', exist_ok=True)
    os.makedirs(VALIDATION_PATH + '/test', exist_ok=True)
    
    timeseries.to_csv(VALIDATION_PATH + '/train/{}.csv'.format(sp), index=False)
    
    test_df = pd.DataFrame(np.array(test))
    test_df.to_csv(VALIDATION_PATH + '/test/{}.csv'.format(sp), index=False)
    
    print('min timepoints:', min(timeseries.apply(count_nonnan, axis=1)), ', # of short timeseries:', short)
    return True
        

with mp.Pool() as pool:
    list(tqdm(pool.imap(cut, enumerate(sources.items())), total=len(sources)))